# odbc Walkthrough

A config file is set up to identify the database server/endpoint and login credentials. The config file can be adjusted to support credentials established via dsn. Available database drivers can be reviewed using the pyodbc drivers() method (ex: pyodbc.drivers()). odbc.py is currently written to support Microsoft SQL Server databases/syntax, but can/will be extended to other databases. ODBC reads in the config file from the current working directory to establish a database connection and return a cursor.

In [37]:
import odbc

#Cursor
config = 'config.json'
curs = odbc.ODBC(config)

Pass a string with SQL syntax to the py_sql method to execute a sql query. The method autocommits queries. This returns a pandas dataframe object.

In [33]:
df = curs.py_sql('SELECT * FROM playground.test.test_table')
df

,id,name,value,value2,dt,boo,nan
0,1,Chris,20,23.45,2014-10-03,True,1.0
1,2,Fred,43,63.56,2008-08-24,False,NaN
2,3,Mary,34,97.62,1984-03-20,False,NaN
3,4,June,65,7.87,1995-04-21,True,2.0
4,5,Barney,75,51.21,1964-11-09,True,NaN


Perhaps users may want to read in sql from other files.

In [30]:
from pathlib2 import Path

def rd_sql(p):
    '''Reads in file and returns a string of the file content.'''
    with open(p, 'r') as f:
        contents = f.readlines()
        return ' '.join(contents)

#Assumes file is in current working directory and file path is a pathlib2 path.
sql = rd_sql(Path('sql_example.txt'))

In [31]:
df2 = curs.py_sql(sql)
df2

,id,name,value,value2,dt,boo,nan
0,1,Chris,20,23.45,2014-10-03,True,1.0
1,4,June,65,7.87,1995-04-21,True,2.0


It's possible the above may be enough for those seeking to draw on data from a database - or manipulate database characteristics using pure sql strings. Those with the appropriate access may want to create their own databases, schemas, and tables (all of which are hierarchical in-nature) using the python examples below.

In [34]:
#Was able to create database, but can't seem to drop it now. Revisit later and delete this database
#using stack overflow post: https://dba.stackexchange.com/questions/58137/db-owner-unable-to-drop-database-error-615-sql-server

#Need to drop db_example database.

db_name = 'ex_db'
schema  = 'ex_schema'
table   = 'ex_table'

#Creating a database named ex_db. Leaning on the instance above.
curs.create_db(db_name)

#The DB class enables manipulation of databases. Creating a schema for the ex_db database.
db = odbc.DB(config, db_name)
db.create_schema(schema)

#Creating a table for the schema created above.
schm = odbc.Schema(config, db_name, schema)
schm.create_table(table, df) #Name of the table and supply a dataframe to create table schema.

#Inserting values into the table created above.
tab_inst = odbc.Table(config, db_name, schema, table)
tab_inst.ins_new_vals(df)

In [35]:
#Validating table/schema created and data inserted above.
curs.py_sql(f'SELECT * FROM {db_name}.{schema}.{table}')

,id,name,value,value2,dt,boo,nan
0,1,Chris,20,23.45,2014-10-03,True,1.0
1,2,Fred,43,63.56,2008-08-24,False,NaN
2,3,Mary,34,97.62,1984-03-20,False,NaN
3,4,June,65,7.87,1995-04-21,True,2.0
4,5,Barney,75,51.21,1964-11-09,True,NaN


In [39]:
#The c attribute represents the connection. Closing. Ideal to close for any/all open instances.
curs.c.close()